# This is the same as DataExtractionProcessing notebook, but it was used for the case of having the data in one file only

In [ ]:
%pylab inline
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib2
import numpy as np
import os
import os.path
from scipy import misc
from time import strftime,strptime
import re
import math
import MySQLdb

##### Note 1: We can use the Classes instead of reimplementing the functions.
##### Note 2: You can use SQLAlchemy instead of generating MySQL queries from a dataframe to insert data into a database.

In [ ]:
def getimage(url, full_name):
    """
    Downloads the image and returns and object of that image.
    url: image url.
    full_name: path with file name to save the image.
    """
    if os.path.isfile(full_name):
        return mpimg.imread(full_name)
    
    try:
        f = urllib2.urlopen(url)
    except:
        return None
    
    data = f.read()
    with open(full_name, "wb") as code:
        code.write(data)
    return mpimg.imread(full_name)

def getallimages(panda_name, column_name):
    """
    Downloads a group of images from a panda dataframe.
    panda_name: dataframe object.
    column_name: column containing the url.
    """
    downloaded_images = []
    for row in panda_name.loc[panda_name[column_name].notnull(),column_name]:
        for image in row:
            img = getimage(image[column_name],image[column_name].split('/')[-1])
            if not(img is None):
                downloaded_images.append(img)
    return downloaded_images

def klout_getId(screen_name):
    """
    Gets user's klout id if exists.
    screenname: user
    """
    url = 'http://api.klout.com/v2/identity.json/twitter?screenName={0}&key=memp3ncn4qvp6c8guzjcc8dp'.format(screen_name)    
    try:
        return json.load(urllib2.urlopen(url))
    except:
        return None
    
def klout_getScore(klout_id):
    """
    Gets user's klout score if exists.
    
    klout_id: user's klout id.
    """
    url = 'http://api.klout.com/v2/user.json/{0}/score?key=memp3ncn4qvp6c8guzjcc8dp'.format(kloutId)
    try:
        return json.load(urllib2.urlopen(url))
    except:
        return None
    
def extractImageUrl(cell):
    """
    Gets user's klout id if exists.
    cell: extracts tweets image url out of media object.
    """
    if cell is None:
        return None
    return cell[0]['media_url']

def convertDTToDB(cell):
    """
    Converts the time to a MySQL-friendly url.
    cell: datetime string.
    """
    return strftime('%Y-%m-%d %H:%M:%S', strptime(cell,'%Y-%m-%dT%H:%M:%S.000Z'))
def getOldId(cell):
    """
    Get's the ID of tweet or user from gnip format.
    cell: string contatining id.
    """
    return cell.split(':')[2]

def normalizeTweetText(text):
    """
    Normalize tweet text to a MySQL-friendly format.
    text: text string.
    """
    if text is None or type(text) is float:
        return None
    return re.sub(r'[^\w#:@/\.\-\,]', ' ', text)

## Note that if we may put NULL for any of the attributes
## we should remove the single quotation marks from around them.
def ensureDBNull(cell):
    return 'NULL' if cell is None else ("'" + cell + "'")

def getTweetsInsertQuery(tweets):
    """
    Generate insert queries for tweet objects.
    tweets: tweets dataframe.
    """
    query = ""
    for tweet in tweets:
        temp = "REPLACE INTO tweet VALUES('{0}', '{1}', '{2}', '{3}', {4}, '{5}', {6}, {7}, {8});\n".format(tweet[0],
                                     tweet[1],
                                     tweet[2],
                                     tweet[3],
                                     ensureDBNull(tweet[4]),
                                     tweet[5],
                                     ensureDBNull(tweet[6]),
                                     ensureDBNull(tweet[7]),
                                     ensureDBNull(tweet[8]))
        
        query = "{0}{1}".format(query, temp)                         
    return query

def getUsersInsertQuery(users):
    """
    Generate insert queries for user objects.
    users: users dataframe.
    """
    query = ""
    for user in users:
        temp = "REPLACE INTO user VALUES('{0}', '{1}', {2}, {3}, \
          '{4}', '{5}', '{6}', {7}, '{8}');\n".format(user[0],
                                                   user[1],
                                                   ensureDBNull(user[2]),
                                                   user[3],
                                                   user[4],
                                                   user[5],
                                                   user[6],
                                                   ensureDBNull(user[7]),
                                                   user[8])

        query = "{0}{1}".format(query, temp)                
    return query

def getPlacesInsertQuery(places):
    """
    Generate insert queries for place objects.
    places: tweets dataframe.
    """
    query = ""
    for place in places:
        temp = "REPLACE INTO place VALUES ('{0}', '{1}', '{2}', '{3}');\n".format(place[0],
                                                     place[1],
                                                     place[2],
                                                     place[3])

        query = "{0}{1}".format(query, temp)
    return query


def convertNanToNone(text):
    """
    Change any 'nan' string to None.
    text: text string.
    """
    try:
        if str(text) == 'nan':
            return None
    except:
        pass
    return text

def listToStr(cell):
    """
    Convert a list of strings to one string holds all of them.
    cell: list of strings.
    """
    if cell is None:
        return None
    mystr = "";
    for i in cell:
        try:
            mystr = mystr + "," + str(i)
        except:
            print i
    mystr = "[{0}]".format(mystr[1:])
    return mystr
def getDBInstance(host, user, password, db):
    """
    Returns a mysql db object.
    Parameters are obvious.
    """
    return MySQLdb.connect(host,
                         user,
                         password,
                         db)
def extractHashTags(hashTags):
    """
    Extract hashtags out of hashtag objects list.
    hashTags: hashtags list.
    """
    if hashTags is None or hashTags == []:
        return None
    mylist = ''
    for hashtag in hashTags:
        mylist = mylist + ',' + hashtag['text']
    return mylist[1:]

In [ ]:
mydb = getDBInstance('geotwitter.uncg.edu', 'root', 'vJnVubg49U', 'geotwitter')
mycursor = mydb.cursor()

In [ ]:
## Get the data and put it in a panda dataframe
myjson = []
myfile = open("/media/saed/Data/ubuntu data/data.json", 'r')
for i in range(10000):
    myjson.append(json.loads(myfile.readline()))
myfile.close()
mypanda = pd.io.json.json_normalize(myjson)

In [ ]:
mydata = mypanda.copy()

In [ ]:
## Choosing the needed columns, removing duplicates,
## remove all-null rows and renaming the columns
mydata = mydata.drop_duplicates(subset = ['id_str'])
mydata = mydata[['id_str','created_at','text','coordinates.coordinates',
                  'entities.media','lang','user.id_str','user.screen_name',
                  'user.location','user.verified','user.followers_count','user.friends_count',
                  'user.statuses_count','user.created_at','place.id','place.full_name',
                  'place.country_code','place.bounding_box.coordinates']]
mydata = mydata.dropna(axis = 0, how = 'all')
mydata = mydata.loc[mydata['id_str'].notnull()]
mydata = mydata.reset_index(drop = True)
mydata = mydata.rename(index=str, columns={'id_str' : 'tweet.id', 'coordinates.coordinates' : 'tweet.coordinates',
                                  'created_at' : 'tweet.created_at', 'text' : 'tweet.text', 'lang' : 'tweet.lang',
                                  'entities.media' : 'tweet.media', 'user.id_str' : 'user.id',
                                  'place.full_name' : 'place.name', 'place.country_code' : 'place.country',
                                  'place.bounding_box.coordinates' : 'place.polygon'})

In [ ]:
## Normalizing data representation a little
mydata = mydata.applymap(convertNanToNone)
mydata['tweet.created_at'] = mydata['tweet.created_at'].apply(convertDTToDB)
mydata['user.created_at'] = mydata['user.created_at'].apply(convertDTToDB)
mydata['tweet.coordinates'] = mydata['tweet.coordinates'].apply(convertNanToNone)
mydata['tweet.text'] = mydata['tweet.text'].apply(normalizeTweetText)
mydata['user.location'] = mydata['user.location'].apply(normalizeTweetText)
mydata['tweet.media'] = mydata['tweet.media'].apply(extractImageUrl)
mydata

In [ ]:
tweets = mydata[['tweet.id','tweet.created_at','tweet.text','user.id','tweet.coordinates','place.id','tweet.media','tweet.lang']]
tweets

In [ ]:
try:
    mycursor.execute(getTweetsInsertQuery(tweets.values))
    mydb.commit()
except:
    pass

In [ ]:
users = mydata[['user.id','user.screen_name','user.location','user.verified','user.followers_count','user.friends_count','user.statuses_count','user.created_at']]
users = users.drop_duplicates(subset = ['user.id'])
users['klout_score'] = [0.0] * len(users)
users

In [ ]:
try:
    mycursor.execute(getUsersInsertQuery(users.values))
    mydb.commit()
except:
    pass

In [ ]:
places = mydata[['place.id','place.name','place.country','place.polygon']]
places = places.drop_duplicates(subset = ['place.id'])
places['place.name'] = places['place.name'].apply(normalizeTweetText)
places

In [ ]:
try:
    mycursor.execute(getPlacesInsertQuery(places.values))
    mydb.commit()
except:
    pass